In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install semanticscholar

In [ ]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=84a07e5f13c09ad47195dc05ee5d67f8ccae769a2b1691b43ce7a17ab47ac666
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import gzip
import shutil
import json
from pprint import pprint
import gensim
import nltk
import re
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Make sure you have NLTK's stopwords downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec
nltk.download('punkt')
from num2words import num2words
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np


import string
import re
from pprint import pprint

# NLTK

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) #this depends on each language
stop_words.add("rs")
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization


# Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [ ]:
import spacy
from spacy.cli.download import download
download(model="en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
input_file_path = '/content/drive/MyDrive/validation-00000-of-00002.json.gz'
output_file_path = 'validation.json'

In [ ]:
with gzip.open(input_file_path, 'rb') as f_in:
    with open(output_file_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [ ]:
all_data = []  # Initialize an empty list to store all the data

with open(output_file_path, 'r') as file:
    for line in file:
        data = json.loads(line)
        all_data.append(data)

In [ ]:
import csv
import pandas as pd

In [ ]:
headers = all_data[0].keys()
csv_file_path = 'output.csv'

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)

    # Write the header row
    writer.writeheader()

    # Write the data rows
    for data_row in all_data:
        writer.writerow(data_row)


In [ ]:
df = pd.read_csv("/content/output.csv")
df.head()

,id,source,version,added,created,text
0,254721023,s2ag/valid,v1,2022-12-16T16:07:05.559Z,2022-12-14T00:00:00.000Z,Investigation Some Characteristics of Biocompo...
1,254200673,s2ag/valid,v1,2022-12-04T16:21:43.441Z,2022-12-02T00:00:00.000Z,Modelling Techniques for Conjunctive Water Man...
2,252643161,s2ag/valid,v1,2022-10-01T15:18:53.788Z,2022-12-01T00:00:00.000Z,Complete Targets Coverage in Energy Harvesting...
3,244852373,s2ag/valid,v1,2021-12-04T16:21:39.980Z,2022-12-01T00:00:00.000Z,Nonorthogonal Multiple Access Enabled Two-Way ...
4,254774363,s2ag/valid,v1,2022-12-17T16:19:10.635Z,2022-12-14T00:00:00.000Z,106 A CASE OF COMPLETE ATRIOVENTRICULAR BLOCK ...


In [ ]:
df.shape

(111228, 6)

In [ ]:
new_df = df.iloc[:111228]


Splitting corpus id's into batch size of 100

In [ ]:
corpus_ids = new_df['id'].tolist()

batch_size = 100
corpus_id_batches = [corpus_ids[i:i + batch_size] for i in range(0, len(corpus_ids), batch_size)]


In [ ]:
import requests

Making API requests to retrieve information about paperId based on their Corpus IDs using the Semantic Scholar API.

In [ ]:


# Define your API key
api_key = 'EmSo3GjIr26BCid9NLmnyGPxZLUf7FE361X7gLw5'

# Define the API endpoint
api_endpoint = 'https://api.semanticscholar.org/graph/v1/paper/'

# Prepare headers with the API key
headers = {'x-api-key': api_key}

# Initialize an empty list to store API responses
api_responses = []

for batch in corpus_id_batches:
    # Construct the full URL with the current Corpus ID
    full_urls = [api_endpoint + 'CorpusID:' + str(corpus_id) for corpus_id in batch]
    batch_responses = [requests.get(full_url, headers=headers) for full_url in full_urls]
    for response in batch_responses:
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            api_response = response.json()  # Extract the JSON response

            # Include 'CorpusID' in the 'api_response' dictionary
            corpus_id = int(response.url.split(':')[-1])
            api_response['CorpusID'] = corpus_id

            # Append the 'api_response' dictionary to the list
            api_responses.append(api_response)



# Create a DataFrame from the list of 'api_responses'
paper_df = pd.DataFrame(api_responses, columns=['paperId', 'CorpusID', 'title'])

# Now 'df' is a DataFrame containing 'paperId', 'CorpusID', and 'title' columns


In [ ]:
paper_df.head()
paper_df = paper_df.rename(columns={'CorpusID': 'id'})

In [ ]:
paper_ids = paper_df['paperId'].tolist()
batch_size = 500
paper_id_batches = [paper_ids[i:i + batch_size] for i in range(0, len(paper_ids), batch_size)]

Makes batch API requests to Semantic Scholar to retrieve 'fieldsOfStudy' information for a list of papers using their paper IDs

In [ ]:
import requests
import json

# Initialize an empty list to store JSON responses
json_data = []

# Iterate through paper_id_batches and make API requests
for paper_id_batch in paper_id_batches:
    r = requests.post(
        'https://api.semanticscholar.org/graph/v1/paper/batch',
        params={'fields': 'fieldsOfStudy'},
        json={"ids": paper_id_batch}
    )

    # Check if the response is successful (status code 200)
    if r.status_code == 200:
        json_response = r.json()
        json_data.extend(json_response)
    else:
        print(f"API request failed with status code: {r.status_code}")
# Print the JSON data for reference
print(json.dumps(json_data, indent=2))


[
  {
    "paperId": "83c78b5e308a1e66f5a20ee925c2d559f89e7f6b",
    "fieldsOfStudy": [
      "Physics"
    ]
  },
  {
    "paperId": "9c402f5d74915a55c21ad7204701da4977a15de0",
    "fieldsOfStudy": null
  },
  {
    "paperId": "14f426031e8b0e672e1acf21bf12aa4ef2434624",
    "fieldsOfStudy": null
  },
  {
    "paperId": "dce33b7cdfb15bead0738d7b7a37054ad0dde148",
    "fieldsOfStudy": [
      "Computer Science"
    ]
  },
  {
    "paperId": "8283ee37589de6744bca081f97c330f7d0933dbd",
    "fieldsOfStudy": null
  },
  {
    "paperId": "423a31c943ccae754b07299c03c441a2277e0a91",
    "fieldsOfStudy": null
  },
  {
    "paperId": "34f57cbe521c8fae2c723f707e61d79301db01dd",
    "fieldsOfStudy": [
      "Computer Science"
    ]
  },
  {
    "paperId": "db168c0cb22faa7fc840381c6e5bebfc6607bd81",
    "fieldsOfStudy": null
  },
  {
    "paperId": "646ac075505e4221f1b45cd8c71314eb963fdf6c",
    "fieldsOfStudy": null
  },
  {
    "paperId": "1577c68372d6ad3a0b568d8a2ffb935a76bf9127",
    "fieldsOfS

Creating dataframe of paperId and fieldOfStudy

In [ ]:
data_list = []

# Iterate through json_data and extract "paperId" and "fieldsOfStudy"
for item in json_data:
    paper_id = item.get("paperId", None)
    fields_of_study = item.get("fieldsOfStudy", None)
    data_list.append({"paperId": paper_id, "fieldsOfStudy": fields_of_study})

# Create a DataFrame
domain_df = pd.DataFrame(data_list)
domain_df = domain_df.dropna(subset=['fieldsOfStudy'])
domain_df['fieldsOfStudy'] = domain_df['fieldsOfStudy'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

domain_df.head()

,paperId,fieldsOfStudy
0,83c78b5e308a1e66f5a20ee925c2d559f89e7f6b,Physics
3,dce33b7cdfb15bead0738d7b7a37054ad0dde148,Computer Science
6,34f57cbe521c8fae2c723f707e61d79301db01dd,Computer Science
9,1577c68372d6ad3a0b568d8a2ffb935a76bf9127,Physics
14,46a76b1451de0baf658a984a48ae09db91a72e0a,Physics


Merging dataframe

In [ ]:
merged_df = paper_df.merge(domain_df, on='paperId', how='inner')
final_df = merged_df.merge(new_df, on='id', how='inner')
final_df.head()

,paperId,id,title,fieldsOfStudy,source,version,added,created,text
0,83c78b5e308a1e66f5a20ee925c2d559f89e7f6b,254205437,Effect of yttrium content on microstructure an...,Physics,s2ag/valid,v1,2022-12-04T16:35:32.346Z,2022-12-02T00:00:00.000Z,Effect of Y content on microstructure and tens...
1,dce33b7cdfb15bead0738d7b7a37054ad0dde148,255088003,A New Measure for Determining the Equivalent S...,Computer Science,s2ag/valid,v1,2022-12-25T16:11:07.508Z,2022-12-23T00:00:00.000Z,A New Measure for Determining the Equivalent S...
2,34f57cbe521c8fae2c723f707e61d79301db01dd,254737490,Neural Network-Based Adaptive Asymptotic Presc...,Computer Science,s2ag/valid,v1,2022-12-17T14:17:55.118Z,2023-01-01T00:00:00.000Z,Neural Network-Based Adaptive Asymptotic Presc...
3,1577c68372d6ad3a0b568d8a2ffb935a76bf9127,255283774,"Visualization, registration and tracking techn...",Physics,s2ag/valid,v1,2022-12-31T16:14:50.124Z,2022-12-29T00:00:00.000Z,"Visualization, registration and tracking techn..."
4,46a76b1451de0baf658a984a48ae09db91a72e0a,255058341,Etched Silicon Planar CRL Optics for the High-...,Physics,s2ag/valid,v1,2022-12-24T20:01:56.485Z,2022-12-01T00:00:00.000Z,Etched Silicon Planar CRL Optics for the High-...


Retrieving paper of 'Computer Science' domain

In [ ]:
filtered_df = final_df[final_df['fieldsOfStudy'] == 'Computer Science']


In [ ]:
filtered_df.to_csv('cs_domain.csv', index=False)